This notebook outlines the process of reading data from an AWS S3 bucket into Databricks. The dataset is stored in the S3 bucket and contains JSON objects organised by paths. It will create three DataFrames for different types of data: df_pin for Pinterest post data, df_geo for geolocation data, and df_user for user data.


In [ ]:
dbutils.fs.ls("/FileStore/tables")

Out[4]: [FileInfo(path='dbfs:/FileStore/tables/authentication_credentials.csv', name='authentication_credentials.csv', size=202, modificationTime=1687110999000)]

In [ ]:
# pyspark functions
from pyspark.sql.functions import *
# URL processing
import urllib

In [ ]:
# Specify file type to be csv
file_type = "csv"
# Indicates file has first row as the header
first_row_is_header = "true"
# Indicates file has comma as the delimeter
delimiter = ","
# Read the CSV file to spark dataframe
aws_keys_df = spark.read.format(file_type)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load("/FileStore/tables/authentication_credentials.csv")

In [ ]:
# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

In [ ]:
# AWS S3 bucket name
AWS_S3_BUCKET = "user-0a48d8473ced-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/user-0a48d8473ced-bucket"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

Out[8]: True

In [ ]:
display(dbutils.fs.ls("/mnt/user-0a48d8473ced-bucket/../.."))

path,name,size,modificationTime
dbfs:/FileStore/,FileStore/,0,1697025807192
dbfs:/Volume/,Volume/,0,0
dbfs:/Volumes/,Volumes/,0,0
dbfs:/databricks-datasets/,databricks-datasets/,0,0
dbfs:/databricks-results/,databricks-results/,0,0
dbfs:/delta/,delta/,0,1697025807192
dbfs:/df_pin.csv/,df_pin.csv/,0,1697025807192
dbfs:/local_disk0/,local_disk0/,0,1697025807192
dbfs:/mnt/,mnt/,0,1697025807192
dbfs:/pin_kinesis_events/,pin_kinesis_events/,0,1697025807192


In [ ]:
# File location and type
# Asterisk(*) indicates reading all the content of the specified file that have .json extension
file_location_pin = "/mnt/user-0a48d8473ced-bucket/topics/0a48d8473ced.pin/partition=0/*.json"
file_location_geo = "/mnt/user-0a48d8473ced-bucket/topics/0a48d8473ced.geo/partition=0/*.json" 
file_location_user = "/mnt/user-0a48d8473ced-bucket/topics/0a48d8473ced.user/partition=0/*.json" 

file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from mounted S3 bucket
df_pin = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location_pin)
df_geo = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location_geo)
df_user = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location_user)

# Display Spark dataframe to check its content
display(df_pin)
display(df_geo)
display(df_user)

category,description,downloaded,follower_count,image_src,index,is_image_or_video,poster_name,save_location,tag_list,title,unique_id
travel,"This Costa Rica itinerary is the ultimate guide to spending two weeks in Costa Rica. Find out about visiting La Fortuna, Arenal, Monteverde, Naranjo, Corcovado National Park, Or…",1,10k,https://i.pinimg.com/originals/30/93/cb/3093cb01d9de2d125fda8ba5e3e41946.jpg,10138,image,"Wanderlust Chloe ✈️ Travel guides, inspo and adventure travel ✈️",Local save in /data/travel,"Costa Rica Travel,Rio Celeste Costa Rica,Dream Vacations,Vacation Spots,Vacation Travel,Travel Pictures,Travel Photos,Fortuna Costa Rica,Costa Rica Pictures","14 Amazing Things To Do In Costa Rica | Volcanoes, Waterfalls, Wildlife And More",927c4658-cc3f-4b92-9b5c-70743d0c238d
diy-and-crafts,"This post may contain affiliate links, read our Disclosure Policy for more information. As an Amazon Associate I earn from qualifying purchases, thank you! Make some cute handpr…",1,892k,https://i.pinimg.com/originals/ff/fe/38/fffe384f3ec18a0d87cb2d80cc8c1499.jpg,3156,image,Michelle {CraftyMorning.com},Local save in /data/diy-and-crafts,"Christmas Gifts For Parents,Christmas Decorations For Kids,Christmas Crafts For Toddlers,Preschool Christmas,Christmas Crafts For Gifts,Christmas Activities,Toddler Crafts,Kids Christmas,Christmas Feeling",Handprint Reindeer Ornaments - Crafty Morning,fa6e31a4-18c2-4eca-a6d8-e903eee2c2a4
finance,"If you love budgeting, make sure to give Dave Ramsey's 7 Baby Steps a try. Follow these steps to begin your debt snowball, build an emergency fund, invest and reach riches. I ca…",1,26k,https://i.pinimg.com/originals/1e/9d/90/1e9d906e4e150e3b95187f3b76ea7c71.png,5494,image,"Living Low Key | Save Money, Make Money, & Frugal Living",Local save in /data/finance,"Financial Peace,Financial Tips,Saving Money Quotes,Total Money Makeover,Budgeting Finances,Money Management,Wealth Management,Personal Finance,Making Ideas",Dave Ramsey's 7 Baby Steps: What Are They And Will They Work For You,8fb2af68-543b-4639-8119-de33d28706ed
christmas,Here are the best DIY Christmas Centerpieces ideas perfect for your Christmas & holiday season home decor. From Christmas Vignettes to Table Centerpieces.,1,500k,https://i.pinimg.com/originals/aa/6d/0f/aa6d0f44d7c1c96b998cb9aa6c4446b8.png,2418,image,HikenDip,Local save in /data/christmas,"Farmhouse Christmas Decor,Rustic Christmas,Christmas Time,Vintage Christmas,Xmas,Primitive Christmas Crafts,Christmas Vignette,Indoor Christmas Decorations,Diy Christmas Ornaments",100 DIY Christmas Centerpieces You'll Love To Decorate Your Home With For The Christmas Season - Hike n Dip,da8745a6-5160-46c4-877d-181d50a729fd
quotes,summcoco gives you inspiration for the women fashion trends you want. Thinking about a new look or lifestyle? This is your ultimate resource to get the hottest trends. 45 Top Li…,1,306k,https://i.pinimg.com/originals/bb/c0/e6/bbc0e6a797079505f11ac12bcb0b8c66.jpg,7922,image,"Sumcoco | Decor Ideas, Hairstyles, Nails Fashion Advice",Local save in /data/quotes,"Life Quotes Love,Inspirational Quotes About Love,Mood Quotes,Motivational Quotes,Tears Quotes,Quotes About Sadness,Deep Quotes About Life,Quotes Quotes,Quote Life",45 Top Life Quotes School Did Not Teach You,a584581c-1b38-4731-a1cc-f36115ecf229
travel,"Are you traveling to Paris during the summer? Find out what to do in Paris, France during the summer. Fun summertime activities in Paris. Enjoy the incredible outdoors when trav…",1,3k,https://i.pinimg.com/originals/6c/4c/90/6c4c90bba27ebf8c8bfe4c1acfb9f07a.jpg,9979,image,Petite in Paris,Local save in /data/travel,"Torre Eiffel Paris,Tour Eiffel,Picnic In Paris,Hello France,Voyage Europe,Destination Voyage,Beautiful Places To Travel,Travel Aesthetic,Paris Travel",Paris in the Summer. 10 fun things to do in Paris in the Summertime • Petite in Paris,2b2abc85-fc51-481f-8ae6-17681993da28
home-decor,"Holiday mantle decor, Christmas decor, metallic mercury glass style Christmas trees,

country,ind,latitude,longitude,timestamp
Antarctica (the territory South of 60 deg S),2418,-88.4642,-171.061,2022-05-27 11:30:59
Cocos (Keeling) Islands,10794,-89.5236,-154.567,2022-01-01 02:26:50
Central African Republic,2074,-52.3213,-50.11,2019-11-03 05:41:59
Antigua and Barbuda,7922,-88.0974,-172.052,2021-01-27 09:14:19
Dominican Republic,9979,14.9967,-120.682,2018-07-18 19:01:46
Cote d'Ivoire,2923,-84.6302,-164.507,2019-09-08 22:53:09
French Guiana,8304,-28.8852,-164.87,2019-09-13 04:50:29
French Guiana,8304,-28.8852,-164.87,2019-09-13 04:50:29
Cote d'Ivoire,4315,-45.8508,66.1003,2019-12-15 03:51:28
Azerbaijan,5069,-63.0063,-157.474,2021-03-20 09:32:44


age,date_joined,first_name,ind,last_name
21,2015-11-10 09:27:42,Andrea,8731,Alexander
24,2016-03-31 20:56:39,Austin,8887,Rodriguez
36,2015-12-20 16:38:13,Michelle,4315,Prince
32,2017-10-10 20:09:33,Christian,10625,Lang
22,2016-02-11 20:46:04,Jennifer,9672,Hudson
32,2016-04-02 03:51:23,Brittany,1313,Jones
34,2016-12-22 00:02:02,Thomas,10794,Turner
20,2016-01-07 19:49:22,David,2959,Griffith
26,2015-12-20 10:28:00,Brendan,9875,Joseph
21,2016-01-03 15:42:12,Annette,2074,Forbes


In [ ]:
display(dbutils.fs.ls("/mnt/user-0a48d8473ced-bucket/topics/0a48d8473ced.pin/partition=0/"))

path,name,size,modificationTime
dbfs:/mnt/user-0a48d8473ced-bucket/topics/0a48d8473ced.pin/partition=0/0a48d8473ced.pin+0+0000000000.json,0a48d8473ced.pin+0+0000000000.json,444,1696877757000
dbfs:/mnt/user-0a48d8473ced-bucket/topics/0a48d8473ced.pin/partition=0/0a48d8473ced.pin+0+0000000001.json,0a48d8473ced.pin+0+0000000001.json,681,1696877760000
dbfs:/mnt/user-0a48d8473ced-bucket/topics/0a48d8473ced.pin/partition=0/0a48d8473ced.pin+0+0000000002.json,0a48d8473ced.pin+0+0000000002.json,714,1696877763000
dbfs:/mnt/user-0a48d8473ced-bucket/topics/0a48d8473ced.pin/partition=0/0a48d8473ced.pin+0+0000000003.json,0a48d8473ced.pin+0+0000000003.json,640,1696877766000
dbfs:/mnt/user-0a48d8473ced-bucket/topics/0a48d8473ced.pin/partition=0/0a48d8473ced.pin+0+0000000004.json,0a48d8473ced.pin+0+0000000004.json,444,1696877800000
dbfs:/mnt/user-0a48d8473ced-bucket/topics/0a48d8473ced.pin/partition=0/0a48d8473ced.pin+0+0000000005.json,0a48d8473ced.pin+0+0000000005.json,681,1696877803000
dbfs:/mnt/user-0a48d8473ced-bucket/topics/0a48d8473ced.pin/partition=0/0a48d8473ced.pin+0+0000000006.json,0a48d8473ced.pin+0+0000000006.json,444,1697024999000
dbfs:/mnt/user-0a48d8473ced-bucket/topics/0a48d8473ced.pin/partition=0/0a48d8473ced.pin+0+0000000007.json,0a48d8473ced.pin+0+0000000007.json,681,1697025002000
dbfs:/mnt/user-0a48d8473ced-bucket/topics/0a48d8473ced.pin/partition=0/0a48d8473ced.pin+0+0000000008.json,0a48d8473ced.pin+0+0000000008.json,714,1697025004000
dbfs:/mnt/user-0a48d8473ced-bucket/topics/0a48d8473ced.pin/partition=0/0a48d8473ced.pin+0+0000000009.json,0a48d8473ced.pin+0+0000000009.json,640,1697025007000
